In [2]:
from gernerate_data import load_clas_seg_data, load_clas_seg_data_for_test, load_SEG_data_for_test
import tensorflow as tf
from sklearn.metrics import classification_report, auc, roc_curve
from keras.utils.np_utils import *
from keras.callbacks import LearningRateScheduler
from models.MTL_IBA import MTL_IBA, MTL_IBA_h3, MTL_IBA_cross3,MTL_IBA_s2c
from models.MTL_Attention import MTL_Attention_model
# from models.shareLayer import create_pair_model
from sklearn.preprocessing import LabelBinarizer, label_binarize
# from utils.losses import dice_coef_loss, dice_coef, dice_p_bce, dice_p_focal, tversky_loss, focal_loss, focal_tversky, \
#     p_r_f1_iou, generalized_dice_coeff, generalized_dice_loss
from keras.losses import categorical_crossentropy, mean_squared_error, binary_crossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import keras.backend as K
from keras import Model
import utils_paths
import numpy as np
import pickle
import os

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
INIT_LR = 2e-4
EPOCHS =200
batch_size = 8
depth = 3
img_size = 512
Name = "MTL_IBA_cross3_512_2e-4"
GPU = True
target = (img_size, img_size)

if GPU:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"


    
print("------------------------------------------------ Preparing model ------------------------------------------------")
model = MTL_IBA_cross3(img_size, img_size, depth, nClasses=2)
# model = MTL_Attention_model(img_size, img_size, depth, nClasses=2)
model.summary()
# model.load_weights("MTL_IBA_cross3_2.h5", by_name=True)

model.load_weights("MTL_IBA_cross3_512.h5", by_name=True)


------------------------------------------------ Preparing model ------------------------------------------------
Instructions for updating:
Colocations handled automatically by placer.


2022-08-15 09:27:44.078512: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-08-15 09:27:44.125220: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200095000 Hz
2022-08-15 09:27:44.130025: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x68b78d0 executing computations on platform Host. Devices:
2022-08-15 09:27:44.130113: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-08-15 09:27:44.862045: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x6963900 executing computations on platform CUDA. Devices:
2022-08-15 09:27:44.862131: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2022-08-15 09:27:44.863726: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: Tes

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 512, 512, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 256, 256, 64) 0           block1_conv2[0][0]          

In [4]:
# trainX_dir = 'dataset/LE/test/images/'
# trainX_dir = 'dataset/LE-NEW/train/images'
# trainX_dir = 'dataset/Dataset_BUSI_AN/train/images'
trainX_dir = 'dataset/Dataset_BUSI_AN/val/images'
testX_dir = trainX_dir.replace('val', 'test')


train_x,  train_c_y, train_s_y = load_clas_seg_data(trainX_dir, target)
# val_x, val_c_y, val_s_y = load_clas_seg_data(valX_dir, target)
test_x, test_c_y, test_s_y = load_clas_seg_data(testX_dir, target)


lb = LabelBinarizer()
train_c_y = lb.fit_transform(train_c_y)
# val_c_y = lb.fit_transform(val_c_y)
test_c_y = lb.fit_transform(test_c_y)
train_c_y = to_categorical(train_c_y, 2)
# val_c_y = to_categorical(val_c_y, 2)
test_c_y = to_categorical(test_c_y, 2)

In [5]:
# from models.shareLayer import create_pair_model
# from models.MLT_net import MTL_classic, MTL_3

# model = MTL_classic(img_size, img_size, depth, nClasses=2)
# model = MTL_3(img_size, img_size, depth, nClasses=2)
# model = create_pair_model(img_size, img_size, depth, nClasses=2)
# model.summary()


# define callbacks
csv_logger = CSVLogger(Name+'.log')

reduce_lr = ReduceLROnPlateau(monitor='val_segmentation_output_loss', factor=0.2, patience=20, min_lr=1e-8, mode='auto', verbose=1)

checkpoint_period1 = ModelCheckpoint(Name + '-{epoch:03d}-{val_classification_output_acc:.4f}.h5',
                                     monitor='val_classification_output_acc', mode='auto', save_best_only='True')



# define loss and compile the model

opt = Adam(lr=INIT_LR, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.01)

def binary_crossentropy(y_true, y_pred):
    e=1.0
    return K.mean(-(y_true*K.log(y_pred+K.epsilon())+
                    e*(1-y_true)*K.log(1-y_pred+K.epsilon())),
                  axis=-1)

def tversky(y_true, y_pred, smooth=1):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    alpha = 0.5
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)

# smooth 参数防止分母为0
def dice_coef(y_true, y_pred, smooth=1):
    smooth = 0.0005
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return K.mean((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))


from keras.losses import categorical_crossentropy

model.compile(loss={'segmentation_output': categorical_crossentropy, 'seg-out': tversky_loss, "classification_output": binary_crossentropy},
              loss_weights={'segmentation_output': 0.35,'seg-out': 0.35, "classification_output": 0.3},
              optimizer=opt, 
              metrics={'segmentation_output':'accuracy', 'seg-out':dice_coef, 'classification_output':'accuracy'})

# model.compile(loss={'segmentation_output': categorical_crossentropy, 'seg-out': tversky_loss, "classification_output": binary_crossentropy},
#               loss_weights={'segmentation_output': 0.15,'seg-out': 0.15, "classification_output": 0.7},
#               optimizer=opt, 
#               metrics={'segmentation_output':'accuracy', 'seg-out':dice_coef, 'classification_output':'accuracy'})

# # model.compile(loss={'segmentation_output': focal_tversky, "classification_output": binary_crossentropy},
# model.compile(loss={'segmentation_output': 'categorical_crossentropy', "classification_output": binary_crossentropy},
#               loss_weights={'segmentation_output': 0.5, "classification_output": 0.5},
#               optimizer=Adam(lr=1e-3), 
#               metrics={'segmentation_output':['accuracy', dice_coef], 'classification_output':'accuracy'})

# model.compile(loss={'seg_out': 'categorical_crossentropy', 'cls_out': binary_crossentropy},
#               optimizer=Adam(lr=1e-4), metrics={'seg_out':dice_coef, 'cls_out':'accuracy'},
#               loss_weights={'seg_out':0.5, 'cls_out':0.5})



In [6]:
# print("================================================ begin training ================================================")

hist = model.fit(train_x,
                 [train_c_y, train_s_y, train_s_y],
                 batch_size=batch_size,
                 epochs=200,
                 validation_data=(val_x, [val_c_y, val_s_y, val_s_y]),
                 verbose=1,
                 callbacks=[reduce_lr, csv_logger],
                 shuffle=True)

print("================================================ Saving model ================================================")
model_filename = Name + ".h5"
model.save_weights(model_filename)
print('model saved to:', model_filename)




# print("================================================ begin training ================================================")

# hist = model.fit([train_x,train_x],
#                  [train_c_y, train_s_y],
#                  batch_size=batch_size,
#                  epochs=200,
#                  validation_data=([val_x,val_x], [val_c_y, val_s_y]),
#                  verbose=1,
#                  callbacks=[reduce_lr, csv_logger],
#                  shuffle=True)

# print("================================================ Saving model ================================================")
# model_filename = Name + "4.h5"
# model.save_weights(model_filename)
# print('model saved to:', model_filename)


NameError: name 'val_x' is not defined

In [ ]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
# plt.plot(N, hist.history["loss"], label="train_loss")
plt.plot(N, hist.history["classification_output_loss"], label="train_c_loss")
plt.plot(N, hist.history["segmentation_output_loss"], label="train_s_loss")

plt.plot(N, hist.history["classification_output_acc"], label="train_c_acc")
plt.plot(N, hist.history["segmentation_output_acc"], label="train_s_acc")
plt.plot(N, hist.history["seg-out_dice_coef"], label="train_s_dice")

# plt.plot(N, hist.history["val_loss"], label="val_loss")
plt.plot(N, hist.history["val_classification_output_loss"], label="val_c_loss")
plt.plot(N, hist.history["val_segmentation_output_loss"], label="val_s_loss")

plt.plot(N, hist.history["val_classification_output_acc"], label="val_c_acc")
plt.plot(N, hist.history["val_segmentation_output_acc"], label="val_s_acc")
plt.plot(N, hist.history["val_seg-out_dice_coef"], label="val_s_dice")

plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy/DSC")
plt.legend()
plt.savefig(os.path.join('output/', Name+'.png'))


In [5]:
print("------Start predicting------")


predictions_c, predictions_s, predictions_s2 = model.predict(test_x, batch_size=32)


print("------------------------------------------------ Classification testing ------------------------------------------------")

print(classification_report(test_c_y.argmax(axis=1), predictions_c.argmax(axis=1), target_names=lb.classes_, digits=6))

print("------------------------------------------------ Segmentation testing ------------------------------------------------")


# evaluate the model
# loss = model.evaluate(test_x, [test_c_y, test_s_y], verbose=0)
loss, cla_loss, seg_loss, seg_loss2, cla_acc, seg_acc, seg_dice_coef = model.evaluate(test_x, [test_c_y, test_s_y, test_s_y], verbose=0)
print('Test total loss:', loss)
print('Test classification loss:', cla_loss)
print('Test segmentation loss:', seg_loss)

print('Test classification accuracy:', cla_acc)
print('Test segmentation accuracy:', seg_acc)
print('Test segmentation dice_coef:', seg_dice_coef)

preds_c, preds_s, preds_s2 = model.predict(test_x, batch_size=8, verbose=1)
test_mask = test_s_y.flatten()
pred_mask = preds_s2.flatten()
fpr, tpr, thresholds = roc_curve(test_mask, pred_mask, pos_label=1)
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)


------Start predicting------


2022-08-05 10:19:51.385369: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


------------------------------------------------ Classification testing ------------------------------------------------
                  precision    recall  f1-score   support

   bengin_images   0.933333  0.976744  0.954545        43
malignant_images   0.947368  0.857143  0.900000        21

        accuracy                       0.937500        64
       macro avg   0.940351  0.916944  0.927273        64
    weighted avg   0.937939  0.937500  0.936648        64

------------------------------------------------ Segmentation testing ------------------------------------------------
Test total loss: 0.2971489503979683
Test classification loss: 0.5242930594831705
Test segmentation loss: 0.2942817658185959
Test classification accuracy: 0.9375
Test segmentation accuracy: 0.9075498580932617
Test segmentation dice_coef: 0.8946788311004639
64/64 [==============================] - 11s 167ms/step
AUC: 0.9536124352660287


In [6]:
print(test_c_y.argmax(axis=1))
# print(test_c_y.argmax(axis=1))

print(predictions_c.argmax(axis=1))
# print(predictions_c)

prediction = predictions_c.argmax(axis=1)
label = test_c_y.argmax(axis=1)

# # 配对T检验
# from scipy import stats
# ttest = stats.ttest_rel(prediction, label, axis=0, nan_policy='propagate')
# print(ttest)

# 预测结果保存在CSV中
import csv

csvFile = open(Name + ".csv", "w")            #创建csv文件
writer = csv.writer(csvFile)                  #创建写的对象
#先写入columns_name     
writer.writerow(["id","label","prediction","prediction probability","bengin","malignant"]) 

for i in range(len(predictions_c)): 
  
    #找到最大概率对应的索引号，该图片即为该索引号对应的类别
#     max_value,index = t.max(probability,1)
#     class_index = result_(index)
#     probability=np.round(probability.cpu().detach().numpy(),3)
    writer.writerow([i, label[i], prediction[i], predictions_c[i][prediction[i]], predictions_c[i][0], predictions_c[i][1]])
csvFile.close()


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1]


## 只加载分割数据测试每个类别的准确率

In [ ]:
import cv2

testX_dir="dataset/Dataset_BUSI_AN/test_M/images"

test_X, test_Y, test_img_Names, test_img_Names = load_SEG_data_for_test(testX_dir, target=target, shuffle=False)



In [ ]:

# define loss and compile the model

opt = Adam(lr=INIT_LR, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.01)

def binary_crossentropy(y_true, y_pred):
    e=1.0
    return K.mean(-(y_true*K.log(y_pred+K.epsilon())+
                    e*(1-y_true)*K.log(1-y_pred+K.epsilon())),
                  axis=-1)

def tversky(y_true, y_pred, smooth=1):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    alpha = 0.5
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)


model2=Model(inputs=model.inputs, outputs=[model.get_layer('segmentation_output').output, model.get_layer('seg-out').output])

model2.compile(loss={'segmentation_output': categorical_crossentropy, 'seg-out': tversky_loss},
              loss_weights={'segmentation_output': 0.5,'seg-out': 0.5},
              optimizer=opt, 
              metrics={'segmentation_output':'accuracy', 'seg-out':dice_coef})

predictions_s, predictions_s2 = model2.predict(test_X, batch_size=8)
loss, seg_loss, seg_loss2, seg_acc, seg_dice_coef = model2.evaluate(test_X, [test_Y, test_Y], verbose=0)

preds_c, preds_s, preds_s2 = model.predict(test_X, batch_size=8, verbose=1)
test_mask = test_Y[:,:,:,0] - test_Y[:,:,:,1]
print(test_mask.shape)
print(preds_s.shape)
pred_mask = preds_s[:,:,:,0] - preds_s[:,:,:,1]
print(pred_mask.shape)
# def dice_coef(y_true, y_pred, smooth=1):
#     smooth = 0.0005
#     y_true_f = y_true.flatten()
#     y_pred_f = y_pred.flatten()
#     intersection = K.sum(y_true_f * y_pred_f)
#     return K.mean((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

def dice_ratio(y_true, y_pred):
    """
    define the dice ratio
    :param y_pred: segmentation result
    :param y_true: ground truth
    :return:
    """
    y_pred = y_pred.flatten()
    y_pred[y_pred > 0.5] = 1
    y_pred[y_pred <= 0.5] = 0

    y_true = y_true.flatten()
    y_true[y_true > 0.5] = 1
    y_true[y_true <= 0.5] = 0

    same = (y_pred * y_true).sum()

    dice = 2*float(same)/float(y_true.sum() + y_pred.sum())

    return dice

dice = dice_ratio(test_mask, pred_mask)

print('Test segmentation accuracy:', seg_acc)
print('Test segmentation dice_coef:', seg_dice_coef)
print("Dice:", dice)
test_mask = test_Y.flatten()
pred_mask = preds_s.flatten()
fpr, tpr, thresholds = roc_curve(test_mask, pred_mask, pos_label=1)
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)

## 保存预测结果

In [ ]:
from gernerate_data import load_SEG_data_for_test
import cv2
def mask2gray(mask, input_type=None):
    if input_type is 'pred':
        mask0 = mask[:, :, 0]
        mask1 = mask[:, :, 1]
        mask = mask1 - mask0
        print("mask.shape: ", mask.shape)
        #mask = np.argmax(mask, axis=-1)
        # print( mask[mask > 0.5] )
        mask[mask > 0.5] = 255
        mask[mask <= 0.5] = 0
    mask = mask.astype(dtype=np.uint8)
    rst = mask.copy()
    cv2.normalize(mask, rst, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    return rst

print("------------------------------ Saving predictions ------------------------------------")


preds = preds_s
preds_savePath = "predcition/BUSI/ECA_s_0.5" # + Name
img_size = (224, 224)

# 保存之前读取原始图像和文件名列表
# test_X, test_Y, test_img_Names,orignal_imgs = load_SEG_data_for_test(testX_dir, img_size)


# for i in range(test_X.shape[0]):
#     img = test_orignal_images[i]

#     gt = test_Y[i, :, :, 1]
#     gt = mask2gray(gt)

#     pred = preds[i]
#     prediction = mask2gray(pred, input_type='pred')

#     save_img = np.zeros([img_size[0], img_size[1]*3, 3], dtype=np.uint8)
#     save_img[:, 0:img_size[0], :] = img[:, :, ::-1]
#     save_img[:, img_size[0]:img_size[1] * 2, :] = cv2.cvtColor(gt, cv2.COLOR_GRAY2RGB)
#     save_img[:, img_size[0] * 2:img_size[1] * 3, :] = cv2.cvtColor(prediction, cv2.COLOR_GRAY2RGB)

#     savePath = os.path.join(preds_savePath, "joint")

#     if not os.path.exists(savePath):
#         os.makedirs(savePath)

#     cv2.imwrite(savePath + "{0}".format(test_img_Names[i]), save_img)

for i in range(test_X.shape[0]):

    img = orignal_imgs[i]
  
    gt = test_Y[i, :, :, 1]
    gt = mask2gray(gt)
    gt = cv2.cvtColor(gt, cv2.COLOR_GRAY2RGB)
    
    pred = preds[i]
    pred = mask2gray(pred, input_type='pred')
    pred = cv2.cvtColor(pred, cv2.COLOR_GRAY2RGB)
    
    savePath = os.path.join(preds_savePath, 'Images')
    saveMaskPath = os.path.join(preds_savePath, 'Masks')
    savePredPath = os.path.join(preds_savePath, 'Predictions')
    
    if not os.path.exists(savePath):
        os.makedirs(savePath)
    if not os.path.exists(saveMaskPath):
        os.makedirs(saveMaskPath)
    if not os.path.exists(savePredPath):
        os.makedirs(savePredPath)
    
    cv2.imwrite(savePath + "/{0}".format(test_img_Names[i]), img)
    cv2.imwrite(saveMaskPath + "/{0}".format(test_img_Names[i]), gt)
    cv2.imwrite(savePredPath + "/{0}".format(test_img_Names[i]), pred)
